In [1]:
# Importamos las librerías generales
import pandas as pd
import json
import gzip
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# Importamos las bibliotecas de procesamiento de texto y análisis de sentimiento
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Objetivos generales de ETL inicial

- Importar correctamente datos de origen JSON.

- De ser necesario. desanidar datos. 

- Eliminar columnas irrelevantes, manejar valores nulos y duplicados.

- De ser necesario, normalizar tipo de datos.

- Crear la columna "sentiment_analysis" aplicando NLP para el dataframe Reviews.

- Realizar trasforamciones para la correcta utilizacion de los datos por parte de las funciones.

- Guardar el dataset ya procesado en formato Parquet.


# REVIEWS


In [37]:
# Lista para almacenar diccionarios creados a partir de cada línea
lista_user_reviews = []

# Ruta del archivo comprimido en modo lectura
archivo_comprimido = 'datasets/raw_data/user_reviews.json.gz'

# Abrir el archivo comprimido y procesar cada línea
with gzip.open(archivo_comprimido, 'rt', encoding='utf-8') as archivo:
    for linea in archivo:
        try:
            # Utilizar eval para convertir cada línea en un diccionario
            json_data = eval(linea)
            lista_user_reviews.append(json_data)
        except ValueError as e:
            # Manejar errores al intentar evaluar la línea como un diccionario 
            print(f"Error en la línea: {linea}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
reviews = pd.DataFrame(lista_user_reviews)
# Creamos una copia para trabajar
df_user_reviews = reviews.copy()


In [38]:
df_user_reviews.head() # hay datos anidados en reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [39]:
# Usamos json_normalize para extraer la informacion anindada en la columna 'reviews'
df_user_reviews = pd.json_normalize(
    # Utilizamos la lista creada al principio
    data=lista_user_reviews, # Json_normalize solo admite objetos no serializados para funcionar
    record_path='reviews', # Indicamos la columna con los datos anidados
    meta=['user_id', 'user_url'] # Campos que se quieren corsevar
)

In [40]:
# Hay columnas innecesarias
df_user_reviews.columns


Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id', 'user_url'],
      dtype='object')

In [41]:
# Eliminamos columnas innecesarias
df_user_reviews.drop(['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1, inplace=True)

In [42]:
df_user_reviews.head(5) # Observamos el resultado

,item_id,recommend,review,user_id
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,22200,True,It's unique and worth a playthrough.,76561197970982479
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,251610,True,I know what you think when you see this title ...,js41637
4,227300,True,For a simple (it's actually not all that simpl...,js41637


In [43]:
# Observamos si hay valores duplicados y nulos 
print(f"Existen {df_user_reviews.duplicated().sum()} valores duplicados \n")
print(df_user_reviews.isnull().sum())


Existen 874 valores duplicados 

item_id      0
recommend    0
review       0
user_id      0
dtype: int64


In [44]:
# Vemos que no hallamos valores faltantes

# Eliminamos duplicados
df_user_reviews = df_user_reviews.drop_duplicates()

# Volvemos a observar duplicados
print(f"Existen {df_user_reviews.duplicated().sum()} valores duplicados")


Existen 0 valores duplicados


> Vamos a crear la columna "sentiment_analysis" sobre df_user_reviews.review aplicando NLP.

In [45]:
# Inicializar el analizador de sentimientos
sia = SentimentIntensityAnalyzer()

In [46]:
# Función para asignar valores según la escala de sentimiento
def obtener_puntuacion_sentimiento(texto):
    if pd.isnull(texto) or texto == '':
        return 1  # Devolver neutral si está vacío o es NaN
    elif isinstance(texto, str):
        sentimiento = sia.polarity_scores(texto)
        puntuacion_compuesta = sentimiento['compound']
        if puntuacion_compuesta >= -0.05:
            return 2  # Buena puntuación
        elif puntuacion_compuesta <= -0.05:
            return 0  # Mala puntuación
        else:
            return 1
    else: 
        return 1  # Devolver neutral para valores que no son str

In [47]:
# Convertir la columna 'review' a cadena de texto
df_user_reviews['review'] = df_user_reviews['review'].astype(str)


In [48]:

# Aplicar la función obtener_puntuacion_sentimiento a la columna 'review'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(obtener_puntuacion_sentimiento)



In [49]:
# Eliminar la columna 'review' ya que no la necesitamos con 'sentiment_analysis'
df_user_reviews = df_user_reviews.drop(columns=['review'], axis=1)

In [50]:
# Vemos como queda el df
df_user_reviews

,item_id,recommend,user_id,sentiment_analysis
0,1250,True,76561197970982479,2
1,22200,True,76561197970982479,2
2,43110,True,76561197970982479,2
3,251610,True,js41637,2
4,227300,True,js41637,2
...,...,...,...,...
59300,70,True,76561198312638244,2
59301,362890,True,76561198312638244,2
59302,273110,True,LydiaMorley,2
59303,730,True,LydiaMorley,2


# ITEMS

In [13]:
# Lista para almacenar diccionarios creados a partir de cada línea
lista_users_items = []

# Ruta del archivo comprimido en modo lectura
archivo_comprimido_users_items = 'datasets/raw_data/users_items.json.gz'

# Abrir el archivo comprimido y procesar cada línea
with gzip.open(archivo_comprimido_users_items, 'rt', encoding='utf-8') as archivo_users_items:
    for linea in archivo_users_items:
        try:
            # Utilizar eval para convertir cada línea en un diccionario
            json_data = eval(linea)
            lista_users_items.append(json_data)
        except ValueError as e:
            # Manejar errores al intentar evaluar la línea como un diccionario 
            print(f"Error en la línea: {linea}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
items = pd.DataFrame(lista_users_items)
# Creamos una copia para trabajar
df_user_items = items.copy()

In [14]:
df_user_items.head() # nuevamente, tenemos datos anidados en items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [15]:
# Usamos json_normalize para extraer la informacion anindada en la columna 'items'

df_user_items = pd.json_normalize(
    data=lista_users_items, # datos de origen en formato no serializado
    record_path='items', # ruta de datos anidados
    meta=['user_id' ] # campos que se quieren corsevar
)

In [16]:
# Vemos que tenemos columnas inecesarias
df_user_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id'],
      dtype='object')

In [17]:
# Eliminamos columnas innecesarias
df_user_items.drop(['playtime_2weeks', 'item_name'], axis=1, inplace=True)

# Observamos el resultado
df_user_items.head(5)

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
1,20,0,76561197970982479
2,30,7,76561197970982479
3,40,0,76561197970982479
4,50,0,76561197970982479


In [18]:
# Observamos si hay duplicados y nulos 
print(f"Existen {df_user_items.duplicated().sum()} valores duplicados\n")
print(df_user_items.isnull().sum())

Existen 59117 valores duplicados

item_id             0
playtime_forever    0
user_id             0
dtype: int64


In [19]:
# Eliminamos datos duplicados
df_user_items = df_user_items.drop_duplicates()

print(f"Existen {df_user_items.duplicated().sum()} valores duplicados")


Existen 0 valores duplicados


In [20]:
# Eliminar filas donde playtime_forever sea igual a 0
df_user_items = df_user_items[df_user_items['playtime_forever'] != 0]

# GAMES

In [94]:
file_path = r'datasets\raw_data\steam_games.json.gz'

with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Convertir la lista de objetos JSON en un DataFrame
games = pd.DataFrame(data)

# Crear una copia para trabajar
df_steam_games = games.copy()


In [78]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# Vemos que tenemos que lidear con una cantidad de NaN muy alta
df_steam_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [80]:
# Imputar semejante cantidad de fantantes es imposible
df_steam_games.dropna(inplace=True) # eliminamos faltantes y asumimos la utilidad de los datos restantes

In [81]:
# Vemos las columnas para descartar que no son pertinentes
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [82]:
# Vamos a quitar las columnas innecesarias
df_steam_games = df_steam_games.drop(["publisher","app_name","url","reviews_url",
    "specs","early_access"], axis=1)

In [83]:
# El id corresponde a item_id en df_users_items, por tanto homologamos al nombre de dicho df
df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})


In [84]:
# Quitamos los corchetes de genres para que la funcion pueda consumir la informacion correctamente

# Lista de columnas a procesar
cols = ['genres']

# Itera a través de las columnas
for i in cols:
    # Convierte los valores de la columna en cadenas de texto
    df_steam_games[i] = df_steam_games[i].astype(str)
    
    # Elimina los corchetes '[' ']' de los valores de la columna
    df_steam_games[i] = df_steam_games[i].str.replace('[', '').str.replace(']', '')


In [85]:
df_steam_games

,genres,title,release_date,tags,price,item_id,developer
88310,"'Action', 'Casual', 'Indie', 'Simulation', 'St...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
88311,"'Free to Play', 'Indie', 'RPG', 'Strategy'",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
88312,"'Casual', 'Free to Play', 'Indie', 'Simulation...",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
88313,"'Action', 'Adventure', 'Casual'",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
88315,"'Action', 'Adventure', 'Simulation'",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...
120439,"'Action', 'Adventure', 'Casual', 'Indie'",Kebab it Up!,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",1.99,745400,Bidoniera Games
120440,"'Casual', 'Indie', 'Simulation', 'Strategy'",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS"""
120441,"'Casual', 'Indie', 'Strategy'",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada
120442,"'Indie', 'Racing', 'Simulation'",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich


In [86]:
# Creamos la columna release_year para trabajar unicamente con el año
df_steam_games["release_year"] = df_steam_games["release_date"]

# Eliminamos la columna original 
del df_steam_games["release_date"]


# Convertir la columna "release_date" al formato de fecha, permitiendo que Pandas infiera el formato
df_steam_games["release_year"] = pd.to_datetime(df_steam_games["release_year"], format='mixed', errors='coerce')

#  Colocar solo el año de la fecha en "release_date" 
df_steam_games["release_year"] = df_steam_games["release_year"].dt.year

In [87]:
# El parametro errors='coerce' convierte en nulos los datos que no se pueden inferir.

# Contamos los nulos resultantes
df_steam_games["release_year"].isnull().sum()

2

In [88]:
# Los eliminamos ya que no representan una cantidad significativa
df_steam_games.dropna(inplace=True) 


In [89]:
# Cambiamos los valores de "release_date" a tipo entero
df_steam_games["release_year"] = df_steam_games["release_year"].astype(int)

In [90]:
# Vamos a pasar los str que refieran a juesgos gratuitos a 0.00 
# y luego pasar todos los precios a float

# Función para convertir los valores de la columna "price"
def convertir_precio(valor):
    try:
        # Intentar convertir a flotante
        return float(valor)
    except ValueError:
        # Si no se puede convertir, verificar si es "Free To Play"
        if "Free" in valor:
            return 0.0
        else:
            # Si no es ninguno de los casos anteriores, asignar 0.0
            return 0.0

# Aplicar la función a la columna "price" del DataFrame
df_steam_games['price'] = df_steam_games['price'].apply(convertir_precio)


In [91]:
df_steam_games.head()

,genres,title,tags,price,item_id,developer,release_year
88310,"'Action', 'Casual', 'Indie', 'Simulation', 'St...",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018
88311,"'Free to Play', 'Indie', 'RPG', 'Strategy'",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980,Secret Level SRL,2018
88312,"'Casual', 'Free to Play', 'Indie', 'Simulation...",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290,Poolians.com,2017
88313,"'Action', 'Adventure', 'Casual'",弹炸人2222,"[Action, Adventure, Casual]",0.99,767400,彼岸领域,2017
88315,"'Action', 'Adventure', 'Simulation'",Battle Royale Trainer,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540,Trickjump Games Ltd,2018


In [21]:
# Finalemente exportamos todos los datos limpios

# Convertir los DataFrame  Pandas a objetos de tabla de Arrow
tabla_arrow1 = pa.Table.from_pandas(df_steam_games)
tabla_arrow2 = pa.Table.from_pandas(df_user_reviews)
tabla_arrow3 = pa.Table.from_pandas(df_user_items)

# Especificar el nombre de los archivos Parquet
archivo_parquet1 = 'datasets/processed_data/steam_games.parquet'
archivo_parquet2 = 'datasets/processed_data/user_reviews.parquet'
archivo_parquet3 = 'datasets/processed_data/user_items.parquet'

# Escribir las tablas Arrow en formato Parquet
pq.write_table(tabla_arrow1, archivo_parquet1)
pq.write_table(tabla_arrow2, archivo_parquet2)
pq.write_table(tabla_arrow3, archivo_parquet3)


> ## Resultado final:

**df_steam_games:** Contiene información sobre juegos de Steam, incluyendo nombre, item_id, año de lanzamiento, precio,  desarrollador y género.

**df_user_items:** Registra el tiempo de juego de los usuarios para cada item.

**df_user_reviews:** Contiene información sobre las reseñas de los usuarios, incluyendo item_id, user_id, recomendación (True/False) y análisis del sentimiento (1=negativo,0=neutro,2=positivo).